In [347]:
# Para trabajar con regex
# -----------------------------------------------------------------------
import re

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os


# Para trabajar con los DataFrames
# -----------------------------------------------------------------------
import pandas as pd

# Para poner barras progreso
# -----------------------------------------------------------------------
from tqdm import tqdm



# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Ignorar warings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Librerias para generar gráficas
# ----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


In [348]:
!pip install openpyxl

### Cargamos los archivos xlsx de paro por municipios y leemos los datos que nos interesan

In [349]:
file_path_new = "datos/estadistica_paro_municipios.xlsx"
excel_data_new = pd.ExcelFile(file_path_new)
excel_data_new.sheet_names


['PORTADA',
 'Indice',
 'PARO A CORUÑA',
 'CONTRATOS CORUÑA',
 'PARO ALBACETE',
 'CONTRATOS ALBACETE',
 'PARO ALICANTE',
 'CONTRATOS ALICANTE',
 'PARO ALMERIA',
 'CONTRATOS ALMERIA',
 'PARO ALAVA',
 'CONTRATOS ARABA',
 'PARO ASTURIAS',
 'CONTRATOS ASTURIAS',
 'PARO AVILA',
 'CONTRATOS AVILA',
 'PARO BADAJOZ',
 'CONTRATOS BADAJOZ',
 'PARO BARCELONA',
 'CONTRATOS BARCELONA',
 'PARO BIZKAIA',
 'CONTRATOS BIZKAIA',
 'PARO BURGOS',
 'CONTRATOS BURGOS',
 'PARO CACERES',
 'CONTRATOS CACERES',
 'PARO CADIZ',
 'CONTRATOS CADIZ',
 'PARO CANTABRIA',
 'CONTRATOS CANTABRIA',
 'PARO CASTELLON',
 'CONTRATOS CASTELLON',
 'PARO CEUTA',
 'CONTRATOS CEUTA',
 'PARO CIUDAD REAL',
 'CONTRATOS CIUDAD REAL',
 'PARO CORDOBA',
 'CONTRATOS CORDOBA',
 'PARO CUENCA',
 'CONTRATOS CUENCA',
 'PARO GIPUZCOA',
 'CONTRATOS GIPUZCOA',
 'PARO GIRONA',
 'CONTRATOS GIRONA',
 'PARO GRANADA',
 'CONTRATOS GRANADA',
 'PARO GUADALAJARA',
 'CONTRATOS GUADALAJARA',
 'PARO HUELVA',
 'CONTRATOS HUELVA',
 'PARO HUESCA',
 'CONTRATOS H

In [350]:
"""
# Cargar la hoja "PARO ALBACETE" en un DataFrame
df_paro_albacete = pd.read_excel(file_path_new, sheet_name="PARO ALBACETE")

# Mostrar las primeras filas del DataFrame para revisión
df_paro_albacete.head()
"""


'\n# Cargar la hoja "PARO ALBACETE" en un DataFrame\ndf_paro_albacete = pd.read_excel(file_path_new, sheet_name="PARO ALBACETE")\n\n# Mostrar las primeras filas del DataFrame para revisión\ndf_paro_albacete.head()\n'

In [351]:
# Limpiar el DataFrame eliminando filas vacías y reestableciendo los encabezados
df_paro_albacete_cleaned = pd.read_excel(file_path_new, sheet_name="PARO ALBACETE", header=4)

# Mostrar las primeras filas para verificar la estructura después de limpiar
df_paro_albacete_cleaned.head()


,Unnamed: 0,MUNICIPIOS,TOTAL,SEXO Y EDAD,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,SECTORES,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,HOMBRES,NaN,NaN,MUJERES,NaN,NaN,AGRI- CULTURA,INDUS- TRIA,CONS- TRUCCIÓN,SERVICIOS,SIN EMPLEO ANTERIOR
1,NaN,NaN,NaN,<25,25 - 44,>=45,<25,25 - 44,>=45,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2001.0,ABENGIBRE,28,,<5,8,<5,5,10,<5,<5,<5,20,<5
4,2002.0,ALATOZ,39,<5,<5,11,<5,8,15,<5,<5,8,26,<5


In [352]:
# Unificar encabezados para simplificar la estructura
df_paro_albacete_cleaned.columns = [
    "Código", "Municipios", "Total", "Hombres <25", "Hombres 25-44", "Hombres >=45",
    "Mujeres <25", "Mujeres 25-44", "Mujeres >=45", "Agricultura", "Industria",
    "Construcción", "Servicios", "Sin Empleo Anterior"
]

# Eliminar filas con datos no relacionados (encabezados adicionales, NaN innecesarios)
df_paro_albacete_cleaned = df_paro_albacete_cleaned.dropna(subset=["Municipios"]).reset_index(drop=True)

df_paro_albacete_cleaned.head()


,Código,Municipios,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
0,2001.0,ABENGIBRE,28,,<5,8,<5,5,10,<5,<5,<5,20,<5
1,2002.0,ALATOZ,39,<5,<5,11,<5,8,15,<5,<5,8,26,<5
2,2003.0,ALBACETE,11185,458,1540,2173,439,2529,4046,517,1008,620,8028,1012
3,2004.0,ALBATANA,63,<5,<5,13,0,18,28,13,5,<5,39,<5
4,2005.0,ALBOREA,58,<5,5,21,<5,10,20,15,6,<5,31,<5


In [353]:
# Crear un diccionario para almacenar los DataFrames de todas las provincias
provinces_data = {}

# Identificar las hojas de interés relacionadas con "PARO"
paro_sheets = [sheet for sheet in excel_data_new.sheet_names if sheet.startswith("PARO")]

# Procesar cada hoja de "PARO" con encabezados
for sheet in paro_sheets:
    # Cargar la hoja correspondiente
    df_sheet = pd.read_excel(file_path_new, sheet_name=sheet, header=4)
    
    # Estandarizar los nombres de las columnas
    df_sheet.columns = [
        "Código", "Municipios", "Total", "Hombres <25", "Hombres 25-44", "Hombres >=45",
        "Mujeres <25", "Mujeres 25-44", "Mujeres >=45", "Agricultura", "Industria",
        "Construcción", "Servicios", "Sin Empleo Anterior"
    ]
    
    # Eliminar filas innecesarias y limpiar el DataFrame
    df_sheet = df_sheet.dropna(subset=["Municipios"]).reset_index(drop=True)
    
    # Guardar el DataFrame procesado en el diccionario
    provinces_data[sheet] = df_sheet

# Mostrar uno de los DataFrames para verificar el proceso
dataframe=provinces_data["PARO ALICANTE"]
dataframe.head()


,Código,Municipios,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
0,3001.0,ADSUBIA,24,<5,5,<5,0,5,10,<5,<5,,21,0
1,3002.0,AGOST,370,14,57,106,11,78,104,42,59,37,205,27
2,3003.0,AGRES,31,<5,5,13,0,5,6,<5,<5,<5,24,
3,3004.0,AIGÜES,70,<5,6,18,0,20,23,,<5,5,57,5
4,3005.0,ALBATERA,948,21,114,176,28,234,375,87,168,55,561,77


In [354]:
# Crear una lista para almacenar los DataFrames con la columna de provincia añadida
combined_data = []

# Iterar por cada provincia en el diccionario
for province, df in provinces_data.items():
    # Extraer el nombre de la provincia del nombre de la hoja
    province_name = province.replace("PARO ", "").strip()
    # Añadir una columna para la provincia
    df["Provincia"] = province_name
    # Añadir el DataFrame a la lista
    combined_data.append(df)

# Combinar todos los DataFrames en uno solo con los datos de paro de todos los municipios
df_paro_municipios = pd.concat(combined_data, ignore_index=True)

# Mostrar el DataFrame combinado
df_paro_municipios.head()


,Código,Municipios,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior,Provincia
0,15001.0,ABEGONDO,198,7,29,47,<5,37,74,8,26,12,139,13,A CORUÑA
1,15002.0,AMES,1487,26,234,338,35,369,485,22,102,88,1120,155,A CORUÑA
2,15003.0,ARANGA,55,<5,8,25,,15,6,<5,<5,<5,44,<5,A CORUÑA
3,15004.0,ARES,310,5,36,103,7,60,99,10,33,37,208,22,A CORUÑA
4,15005.0,ARTEIXO,1862,44,279,431,28,423,657,27,229,141,1311,154,A CORUÑA


In [355]:
df_paro_municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8186 entries, 0 to 8185
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Código               8134 non-null   float64
 1   Municipios           8186 non-null   object 
 2   Total                8186 non-null   object 
 3   Hombres <25          8186 non-null   object 
 4   Hombres 25-44        8186 non-null   object 
 5   Hombres >=45         8186 non-null   object 
 6   Mujeres <25          8186 non-null   object 
 7   Mujeres 25-44        8186 non-null   object 
 8   Mujeres >=45         8186 non-null   object 
 9   Agricultura          8186 non-null   object 
 10  Industria            8186 non-null   object 
 11  Construcción         8186 non-null   object 
 12  Servicios            8186 non-null   object 
 13  Sin Empleo Anterior  8186 non-null   object 
 14  Provincia            8186 non-null   object 
dtypes: float64(1), object(14)
memory usage

In [356]:
# Situamos la columna 'Provincia' a continuación de 'Municipios'
columns = df_paro_municipios.columns.tolist()
columns.remove('Provincia')
municipios_index = columns.index('Municipios')
columns.insert(municipios_index + 1, 'Provincia')
df_paro_municipios = df_paro_municipios[columns]
df_paro_municipios.head()

,Código,Municipios,Provincia,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
0,15001.0,ABEGONDO,A CORUÑA,198,7,29,47,<5,37,74,8,26,12,139,13
1,15002.0,AMES,A CORUÑA,1487,26,234,338,35,369,485,22,102,88,1120,155
2,15003.0,ARANGA,A CORUÑA,55,<5,8,25,,15,6,<5,<5,<5,44,<5
3,15004.0,ARES,A CORUÑA,310,5,36,103,7,60,99,10,33,37,208,22
4,15005.0,ARTEIXO,A CORUÑA,1862,44,279,431,28,423,657,27,229,141,1311,154


In [357]:
df_paro_municipios.to_csv("datos/paro_municipios_limpiados.csv", index=False)

In [358]:
df_paro_municipios = pd.read_csv("datos/paro_municipios_limpiados.csv")
df_paro_municipios.head()

,Código,Municipios,Provincia,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
0,15001.0,ABEGONDO,A CORUÑA,198,7,29,47,<5,37,74,8,26,12,139,13
1,15002.0,AMES,A CORUÑA,1487,26,234,338,35,369,485,22,102,88,1120,155
2,15003.0,ARANGA,A CORUÑA,55,<5,8,25,,15,6,<5,<5,<5,44,<5
3,15004.0,ARES,A CORUÑA,310,5,36,103,7,60,99,10,33,37,208,22
4,15005.0,ARTEIXO,A CORUÑA,1862,44,279,431,28,423,657,27,229,141,1311,154


In [359]:
df_paro_municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8186 entries, 0 to 8185
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Código               8134 non-null   float64
 1   Municipios           8186 non-null   object 
 2   Provincia            8186 non-null   object 
 3   Total                8186 non-null   object 
 4   Hombres <25          8186 non-null   object 
 5   Hombres 25-44        8186 non-null   object 
 6   Hombres >=45         8186 non-null   object 
 7   Mujeres <25          8186 non-null   object 
 8   Mujeres 25-44        8186 non-null   object 
 9   Mujeres >=45         8186 non-null   object 
 10  Agricultura          8186 non-null   object 
 11  Industria            8186 non-null   object 
 12  Construcción         8186 non-null   object 
 13  Servicios            8186 non-null   object 
 14  Sin Empleo Anterior  8186 non-null   object 
dtypes: float64(1), object(14)
memory usage

Vamos a asegurarnos que no tengamos municpios duplicados.

In [360]:
# Obtener el número de valores únicos en la columna "Municipios"
num_valores_unicos = df_paro_municipios['Municipios'].nunique()
num_valores_unicos

8129

In [361]:
# Identificar duplicados en la columna "Municipios"
df_duplicados_municipios = df_paro_municipios[df_paro_municipios['Municipios'].duplicated(keep=False)]

# Mostrar los duplicados
df_duplicados_municipios

,Código,Municipios,Provincia,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
76,15075.0,SADA,A CORUÑA,854,12,117,208,17,178,322,12,79,59,630,74
81,15080.0,SOBRADO,A CORUÑA,47,0,8,15,<5,13,10,<5,<5,<5,29,6
98,2003.0,ALBACETE,ALBACETE,11185,458,1540,2173,439,2529,4046,517,1008,620,8028,1012
183,NaN,ALBACETE,ALBACETE,26985,926,3119,5325,1002,6095,10518,1916,2970,1454,18706,1939
197,3014.0,ALICANTE/ALACANT,ALICANTE,28876,866,4236,7238,797,6198,9541,561,1400,2626,21482,2807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7859,49241.0,VILLAESCUSA,ZAMORA,11,,,5,<5,<5,<5,<5,,0,9,<5
7890,49275.0,ZAMORA,ZAMORA,4091,153,582,915,147,920,1374,182,279,302,2925,403
7891,NaN,ZAMORA,ZAMORA,9999,371,1313,2510,379,2053,3373,588,746,741,7083,841
8183,50297.0,ZARAGOZA,ZARAGOZA,33267,1278,4473,7361,1200,7494,11461,705,3407,2021,24051,3083


In [362]:
# Filtrar todas las filas donde el valor de la columna "Municipios" sea "SADA"
filas_sada = df_paro_municipios[df_paro_municipios['Municipios'] == "SADA"]

filas_sada


,Código,Municipios,Provincia,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
76,15075.0,SADA,A CORUÑA,854,12,117,208,17,178,322,12,79,59,630,74
5205,31212.0,SADA,NAVARRA,<5,,<5,0,,0,<5,<5,0,0,<5,


Tenemos dos tipos de duplicados en los municipios. En unos, coincide el nombre del municipio, pero no de la provincia, se coresponde con municipios que aunque se llamen igual son localidades diferentes.

Los otros, son aquellos que tienen valor nulo en la columna código y en realidad son los nombres de la provincia. Contienen los datos globales de todos los municipios de la provinicia y coinciden en nombre con un municipio de dicha provincia. Como los datos de paro en las provincias los hemos extraido del INE y tratado en un dataframe aparte, los eliminamos de este.

In [363]:
df_paro_municipios= df_paro_municipios.dropna(subset=['Código'])
df_paro_municipios.reset_index(drop=True, inplace=True)

Comprobamos que ya no tenemos duplicados asociados al mismo municipio

In [364]:
# Identificar duplicados en la columna "Municipios"
df_duplicados_municipios = df_paro_municipios[df_paro_municipios['Municipios'].duplicated(keep=False)]

# Mostrar los duplicados
df_duplicados_municipios

,Código,Municipios,Provincia,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
76,15075.0,SADA,A CORUÑA,854,12,117,208,17,178,322,12,79,59,630,74
81,15080.0,SOBRADO,A CORUÑA,47,0,8,15,<5,13,10,<5,<5,<5,29,6
512,33037.0,MIERES,ASTURIAS,2902,97,473,702,98,596,936,60,228,247,2009,358
922,6118.0,SANCTI-SPIRITUS,BADAJOZ,12,<5,0,<5,<5,<5,5,<5,<5,0,8,<5
967,6162.0,"ZARZA, LA",BADAJOZ,253,13,14,55,12,57,102,30,8,20,182,13
1784,10023.0,ARROYOMOLINOS,CACERES,53,<5,<5,14,,8,26,10,<5,<5,35,<5
1916,10154.0,REBOLLAR,CACERES,7,0,<5,<5,0,<5,<5,<5,,<5,5,0
2051,39021.0,CIEZA,CANTABRIA,20,0,<5,<5,0,10,<5,0,<5,,19,0
2129,39099.0,VILLAESCUSA,CANTABRIA,204,12,32,51,8,36,65,6,11,14,157,16
2162,12033.0,CABANES,CASTELLON,150,<5,20,42,5,39,40,15,28,7,87,13


In [365]:
df_paro_municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8134 entries, 0 to 8133
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Código               8134 non-null   float64
 1   Municipios           8134 non-null   object 
 2   Provincia            8134 non-null   object 
 3   Total                8134 non-null   object 
 4   Hombres <25          8134 non-null   object 
 5   Hombres 25-44        8134 non-null   object 
 6   Hombres >=45         8134 non-null   object 
 7   Mujeres <25          8134 non-null   object 
 8   Mujeres 25-44        8134 non-null   object 
 9   Mujeres >=45         8134 non-null   object 
 10  Agricultura          8134 non-null   object 
 11  Industria            8134 non-null   object 
 12  Construcción         8134 non-null   object 
 13  Servicios            8134 non-null   object 
 14  Sin Empleo Anterior  8134 non-null   object 
dtypes: float64(1), object(14)
memory usage

In [366]:
# Identificar los valores no numéricos en la columna "Total"
df_valores_no_numericos = df_paro_municipios[~df_paro_municipios['Total'].str.isnumeric()]

# Mostrar los valores no numéricos
df_valores_no_numericos

,Código,Municipios,Provincia,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
63,15902.0,OZA-CESURAS,A CORUÑA,,,,,,,,,,,,
144,2050.0,MONTALVOS,ALBACETE,<5,,0,<5,,0,,<5,,,0,
220,3039.0,BENIMASSOT,ALICANTE,<5,,<5,0,,,<5,,,<5,<5,
312,3130.0,TOLLOS,ALICANTE,<5,,<5,<5,,,<5,,,,<5,
355,4033.0,CASTRO DE FILABRES,ALMERIA,<5,,0,<5,,<5,<5,0,,,<5,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8124,50289.0,VILLANUEVA DE JILOCA,ZARAGOZA,<5,,,<5,,<5,,<5,<5,,0,
8125,50291.0,VILLAR DE LOS NAVARROS,ZARAGOZA,,,,,,,,,,,,
8128,50294.0,VILLARROYA DEL CAMPO,ZARAGOZA,,,,,,,,,,,,
8129,50282.0,"VILUEÑA, LA",ZARAGOZA,<5,,<5,,,0,,0,<5,,,


In [367]:
# Obtener los valores únicos de la columna "Total" en el DataFrame de valores no numéricos
df_valores_no_numericos['Total'].unique()


array([' ', '<5', '685.0', '315.0', '146.0', '14108.0', '495.0', '4162.0',
       '2943.0', '2433.0', '803.0', '68.0', '713.0', '194.0', '11165.0',
       '295.0', '9779.0', '2289.0', '1884.0', '310.0', '155.0', '209.0',
       '25563.0', '692.0', '9219.0', '1388.0', '546.0', '716.0', '683.0',
       '9303.0', '4687.0', '420.0', '2684.0', '10394.0', '511.0',
       '8190.0', '229.0', '3287.0', '189.0', '1446.0', '79.0', '609.0',
       '1560.0', '1497.0', '33.0', '1498.0', '92.0', '9858.0', '135.0',
       '663.0', '121.0', '120.0', '800.0', '124.0', '1145.0', '338.0',
       '364.0', '219.0', '73.0', '1385.0', '113.0', '119.0', '85.0',
       '885.0', '376.0', '327.0', '18.0', '31168.0', '201.0', '169.0',
       '163.0', '89.0', '224.0', '649.0', '46.0', '40.0', '593.0',
       '499.0', '26.0', '70.0', '82.0', '53.0', '28.0', '682.0', '285.0',
       '138.0', '3762.0', '154.0', '213.0', '1546.0', '532.0', '110.0',
       '165.0', '187.0', '51.0', '1334.0', '210.0', '125.0', '1397.0',


Los valores no numéricos de la columna "Total" son o valores reales, con parte decimal 0 y por tanto convertibles a enteros o espacios en blanco (" ") o valores <5 que indica "menor que 5". Nos interesa convertir la coluna "Total" a entero así que a continuación vamos a decidir que hacer con los valores que pueden resultar problemáticos al hacerlo.

In [368]:
# Filtrar todas las filas donde el valor de la columna "Total" sea "0"
df_total_cero = df_paro_municipios[df_paro_municipios['Total'] == "0"]
df_total_cero

,Código,Municipios,Provincia,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
249,3068.0,FAMORCA,ALICANTE,0,,0,0,,,,,0,,0,
590,5045.0,CABIZUELA,AVILA,0,,,0,,0,,0,,,0,
607,5062.0,COLLADO DE CONTRERAS,AVILA,0,,0,,,0,,0,,,0,
629,5086.0,GIMIALCON,AVILA,0,,,0,,0,,0,0,,0,
686,5148.0,NARROS DEL PUERTO,AVILA,0,,0,,,0,,,,,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8059,50224.0,RETASCON,ZARAGOZA,0,,0,0,,,,,,,0,
8063,50229.0,RUESCA,ZARAGOZA,0,,0,,,0,,,0,,0,
8073,50238.0,SANTA EULALIA DE GALLEGO,ZARAGOZA,0,,0,,,,0,,,,0,
8074,50239.0,SANTED,ZARAGOZA,0,,,0,,,,,,,0,


In [369]:
# Filtrar las filas donde el valor de la columna "Total" sea menor o igual a 4
df_total_menor_igual_4 = df_paro_municipios[pd.to_numeric(df_paro_municipios['Total'], errors='coerce') <= 4]

# Mostrar las filas filtradas
df_total_menor_igual_4

,Código,Municipios,Provincia,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
249,3068.0,FAMORCA,ALICANTE,0,,0,0,,,,,0,,0,
590,5045.0,CABIZUELA,AVILA,0,,,0,,0,,0,,,0,
607,5062.0,COLLADO DE CONTRERAS,AVILA,0,,0,,,0,,0,,,0,
629,5086.0,GIMIALCON,AVILA,0,,,0,,0,,0,0,,0,
686,5148.0,NARROS DEL PUERTO,AVILA,0,,0,,,0,,,,,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8059,50224.0,RETASCON,ZARAGOZA,0,,0,0,,,,,,,0,
8063,50229.0,RUESCA,ZARAGOZA,0,,0,,,0,,,0,,0,
8073,50238.0,SANTA EULALIA DE GALLEGO,ZARAGOZA,0,,0,,,,0,,,,0,
8074,50239.0,SANTED,ZARAGOZA,0,,,0,,,,,,,0,


Vemos pues que las filas que tienen valores menores o iguales que 4 en la columna "Total" tienen todas valor 0.

Del mismo modo, y teniendo en cuenta que corresponden a municipios de población muy pequeña al convertir la columna "Total" a entero, las celdas de esta columna con valor <5 las imputaremos el valor 4, ya que en el dataframe no tenemos valores numericos para esa columna entre 0 y 5. Estamos cometiendo una sobreestimación del paro, pero a efectos de tomar medidas sobre el paro creemos que es mejor una sobreestimación que una subestimación.

Queda ver como se trataran las celdas cuyo valor "Total" sea une espacio en blanco

In [370]:
# Filtrar las filas donde el valor de la columna "Total" sea un espacio en blanco (' ')
df_total_espacio = df_paro_municipios[df_paro_municipios['Total'] == ' ']

df_total_espacio.sample(20)


,Código,Municipios,Provincia,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
3366,19234.0,RETIENDAS,GUADALAJARA,,,,,,,,,,,,
3416,19288.0,TORTUERO,GUADALAJARA,,,,,,,,,,,,
7513,47151.0,SAN SALVADOR,VALLADOLID,,,,,,,,,,,,
6366,42153.0,RELLO,SORIA,,,,,,,,,,,,
7882,50042.0,BALCONCHAN,ZARAGOZA,,,,,,,,,,,,
3326,19191.0,MONASTERIO,GUADALAJARA,,,,,,,,,,,,
579,5034.0,BLASCONUÑO DE MATACABRAS,AVILA,,,,,,,,,,,,
1486,9120.0,ENCIO,BURGOS,,,,,,,,,,,,
3240,19087.0,CINCOVILLAS,GUADALAJARA,,,,,,,,,,,,
4042,26147.0,TERROBA,LA RIOJA,,,,,,,,,,,,


In [371]:
df_total_espacio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 244 entries, 63 to 8128
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Código               244 non-null    float64
 1   Municipios           244 non-null    object 
 2   Provincia            244 non-null    object 
 3   Total                244 non-null    object 
 4   Hombres <25          244 non-null    object 
 5   Hombres 25-44        244 non-null    object 
 6   Hombres >=45         244 non-null    object 
 7   Mujeres <25          244 non-null    object 
 8   Mujeres 25-44        244 non-null    object 
 9   Mujeres >=45         244 non-null    object 
 10  Agricultura          244 non-null    object 
 11  Industria            244 non-null    object 
 12  Construcción         244 non-null    object 
 13  Servicios            244 non-null    object 
 14  Sin Empleo Anterior  244 non-null    object 
dtypes: float64(1), object(14)
memory usage: 30.

Estas celdas se corresponden a Municipios de pocos habitantes en los que recoger esta clase de información es complicado pero podemos presuponer que la cifra de Parados estará proxima a 0 como en los de similar tamaño de los que si disponemos datos. Así, imputaremos estos valores a 0.

Hacemos las transformaciones que hemos enunciado.

In [372]:
# Reemplazar valores ' ' con 0 y valores '<5' con 4 en la columna "Total"
df_paro_municipios['Total'] = df_paro_municipios['Total'].replace([' '], 0)
df_paro_municipios['Total'] = df_paro_municipios['Total'].replace(['<5'], 4)

Ahora ya, convertimos la columna "Total" a tipo entero.

In [373]:
# Convertir la columna "Total" a tipo entero, manejando si los hubiera, valores no numéricos
df_paro_municipios['Total'] = pd.to_numeric(df_paro_municipios['Total'], errors='coerce').astype('int64')

df_paro_municipios.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8134 entries, 0 to 8133
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Código               8134 non-null   float64
 1   Municipios           8134 non-null   object 
 2   Provincia            8134 non-null   object 
 3   Total                8134 non-null   int64  
 4   Hombres <25          8134 non-null   object 
 5   Hombres 25-44        8134 non-null   object 
 6   Hombres >=45         8134 non-null   object 
 7   Mujeres <25          8134 non-null   object 
 8   Mujeres 25-44        8134 non-null   object 
 9   Mujeres >=45         8134 non-null   object 
 10  Agricultura          8134 non-null   object 
 11  Industria            8134 non-null   object 
 12  Construcción         8134 non-null   object 
 13  Servicios            8134 non-null   object 
 14  Sin Empleo Anterior  8134 non-null   object 
dtypes: float64(1), int64(1), object(13)
me

No hay valores nulos, luego se ha convertido con exito toda la columna "Total"

Procedemos dela misma manera para las columnas  que para la columna "Total"

In [374]:
# Reemplazar valores ' ' con 0 y valores '<5' con 4 en las columnas indicadas
columnas = ["Hombres <25", "Hombres 25-44", "Hombres >=45", "Mujeres <25", "Mujeres 25-44", "Mujeres >=45"]
df_paro_municipios[columnas] = df_paro_municipios[columnas].replace([' '], 0)
df_paro_municipios[columnas] = df_paro_municipios[columnas].replace(['<5'], 4)



In [375]:
df_paro_municipios[columnas] = df_paro_municipios[columnas].apply(pd.to_numeric, errors='coerce').astype('int64')

# Mostrar información del DataFrame
df_paro_municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8134 entries, 0 to 8133
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Código               8134 non-null   float64
 1   Municipios           8134 non-null   object 
 2   Provincia            8134 non-null   object 
 3   Total                8134 non-null   int64  
 4   Hombres <25          8134 non-null   int64  
 5   Hombres 25-44        8134 non-null   int64  
 6   Hombres >=45         8134 non-null   int64  
 7   Mujeres <25          8134 non-null   int64  
 8   Mujeres 25-44        8134 non-null   int64  
 9   Mujeres >=45         8134 non-null   int64  
 10  Agricultura          8134 non-null   object 
 11  Industria            8134 non-null   object 
 12  Construcción         8134 non-null   object 
 13  Servicios            8134 non-null   object 
 14  Sin Empleo Anterior  8134 non-null   object 
dtypes: float64(1), int64(7), object(7)
mem

In [376]:
df_paro_municipios.sample(20)

,Código,Municipios,Provincia,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
4521,25224.0,"TORMS, ELS",LLEIDA,4,0,0,4,4,4,4,<5,0,0,<5,
1326,48040.0,GATIKA,BIZKAIA,55,6,4,13,4,9,23,<5,12,<5,37,<5
7827,49261.0,VILLARALBO,ZAMORA,141,4,22,36,4,27,48,<5,11,7,114,5
4075,26183.0,ZORRAQUIN,LA RIOJA,4,0,4,0,0,0,4,,<5,,<5,
2023,11036.0,TORRE ALHAQUIME,CADIZ,79,12,14,22,7,16,8,12,<5,10,47,9
2693,20007.0,ALTZO,GIPUZCOA,8,0,4,4,0,0,4,,0,<5,7,0
6525,43045.0,COLLDEJOU,TARRAGONA,7,0,4,4,0,4,4,<5,<5,0,5,
3614,22094.0,CHALAMERA,HUESCA,4,0,4,0,0,4,0,0,,<5,<5,
3647,22137.0,LANAJA,HUESCA,41,4,5,15,4,7,11,11,<5,<5,20,<5
718,5181.0,PASCUALCOBO,AVILA,4,0,4,4,0,0,0,,,,<5,


La columna codigo es de tipo float, pero es un id de numeros enteros, por lo que lo pasaremos a tipo entero.

In [377]:
# Convertir la columna "Código" a tipo entero
df_paro_municipios['Código'] = df_paro_municipios['Código'].astype('int64')
df_paro_municipios.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8134 entries, 0 to 8133
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Código               8134 non-null   int64 
 1   Municipios           8134 non-null   object
 2   Provincia            8134 non-null   object
 3   Total                8134 non-null   int64 
 4   Hombres <25          8134 non-null   int64 
 5   Hombres 25-44        8134 non-null   int64 
 6   Hombres >=45         8134 non-null   int64 
 7   Mujeres <25          8134 non-null   int64 
 8   Mujeres 25-44        8134 non-null   int64 
 9   Mujeres >=45         8134 non-null   int64 
 10  Agricultura          8134 non-null   object
 11  Industria            8134 non-null   object
 12  Construcción         8134 non-null   object
 13  Servicios            8134 non-null   object
 14  Sin Empleo Anterior  8134 non-null   object
dtypes: int64(8), object(7)
memory usage: 953.3+ KB


In [378]:
df_paro_municipios.sample(20)

,Código,Municipios,Provincia,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
4636,28018,BECERRIL DE LA SIERRA,MADRID,294,6,41,83,10,51,103,5,9,28,229,23
1550,9206,MAZUELA,BURGOS,0,0,0,0,0,0,0,,,,,
3648,22139,LAPERDIGUERA,HUESCA,4,0,0,0,0,4,0,0,,,<5,
6297,42061,CIDONES,SORIA,7,0,4,0,0,4,4,,,<5,6,
2220,12097,SACAÑET,CASTELLON,4,0,0,4,0,4,4,<5,,,<5,
554,33078,YERNES Y TAMEZA,ASTURIAS,9,0,4,6,0,0,4,,,,9,
4444,25136,"MOLSOSA, LA",LLEIDA,4,0,0,4,0,4,4,,<5,,<5,
5901,37359,VILLAR DE LA YEGUA,SALAMANCA,9,4,0,5,0,0,4,,,,7,<5
2146,12015,ARGELITA,CASTELLON,11,0,4,4,4,4,0,8,<5,,<5,
6894,44265,VINACEITE,TERUEL,7,0,4,4,0,4,4,<5,,,<5,


In [379]:
# Eliminar las columnas especificadas del DataFrame directamente
df_paro_municipios.drop(columns=["Agricultura", "Industria", "Construcción", "Servicios", "Sin Empleo Anterior"], inplace=True)

# Mostrar el DataFrame actualizado
df_paro_municipios.head()


,Código,Municipios,Provincia,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45
0,15001,ABEGONDO,A CORUÑA,198,7,29,47,4,37,74
1,15002,AMES,A CORUÑA,1487,26,234,338,35,369,485
2,15003,ARANGA,A CORUÑA,55,4,8,25,0,15,6
3,15004,ARES,A CORUÑA,310,5,36,103,7,60,99
4,15005,ARTEIXO,A CORUÑA,1862,44,279,431,28,423,657


Para nuestro análisis no nos importa la edad y si saber el numero de hombres y mujeres en paro de cada localidad así pues agrupamos las columaas de hombres en una y lo mismo para las de mujeres

In [380]:
# Agrupar y sumar las columnas de hombres
df_paro_municipios['Hombres'] = (
    df_paro_municipios[['Hombres <25', 'Hombres 25-44', 'Hombres >=45']].sum(axis=1)
)

# Agrupar y sumar las columnas de mujeres
df_paro_municipios['Mujeres'] = (
    df_paro_municipios[['Mujeres <25', 'Mujeres 25-44', 'Mujeres >=45']].sum(axis=1)
)

# Eliminar las columnas individuales de hombres y mujeres para simplificar
df_paro_municipios = df_paro_municipios.drop(
    columns=['Hombres <25', 'Hombres 25-44', 'Hombres >=45', 'Mujeres <25', 'Mujeres 25-44', 'Mujeres >=45']
)

df_paro_municipios.head()

,Código,Municipios,Provincia,Total,Hombres,Mujeres
0,15001,ABEGONDO,A CORUÑA,198,83,115
1,15002,AMES,A CORUÑA,1487,598,889
2,15003,ARANGA,A CORUÑA,55,37,21
3,15004,ARES,A CORUÑA,310,144,166
4,15005,ARTEIXO,A CORUÑA,1862,754,1108


Nótese que para algunos municipios el Total (de parados) no se corresponde con la suma de Hombres y Mujeres. Tengamos en cuenta que hicimos una aproximación en los valores no numéricos de estas tres columnas, por lo que en los municipios pequeños estos datos pueden no ser exactos.

In [381]:

# Usamos la función pd.MultiIndex.from_tuples() para crear encabezados y subencabezados
new_columns = pd.MultiIndex.from_tuples(
    [
        ("Municipio", "Código"),
        ("Municipio", "Nombre del municipio"),
        ("Provincia", "Nombre de la provincia"),
        ("Número de parados", "Total"),
        ("Número de parados", "Hombres"),
        ("Número de parados", "Mujeres"),
    ]
)

# Asignar el nuevo MultiIndex a las columnas del DataFrame
df_paro_municipios.columns = new_columns

# Mostrar el DataFrame actualizado
df_paro_municipios.head()


Municipio                                   Provincia Número de parados  \
     Código Nombre del municipio Nombre de la provincia             Total   
0     15001             ABEGONDO               A CORUÑA               198   
1     15002                 AMES               A CORUÑA              1487   
2     15003               ARANGA               A CORUÑA                55   
3     15004                 ARES               A CORUÑA               310   
4     15005              ARTEIXO               A CORUÑA              1862   

                   
  Hombres Mujeres  
0      83     115  
1     598     889  
2      37      21  
3     144     166  
4     754    1108

In [382]:
# Reordenar las columnas para que "Total" aparezca después de "Hombres y Mujeres"
df_paro_municipios = df_paro_municipios.reorder_levels([0, 1], axis=1)
new_order = [
        ("Municipio", "Código"),
        ("Municipio", "Nombre del municipio"),
        ("Provincia", "Nombre de la provincia"),
        ("Número de parados", "Hombres"),
        ("Número de parados", "Mujeres"),
        ("Número de parados", "Total")
    ]
df_paro_municipios = df_paro_municipios[new_order]

# Mostrar el DataFrame actualizado
df_paro_municipios.head()


Municipio                                   Provincia Número de parados  \
     Código Nombre del municipio Nombre de la provincia           Hombres   
0     15001             ABEGONDO               A CORUÑA                83   
1     15002                 AMES               A CORUÑA               598   
2     15003               ARANGA               A CORUÑA                37   
3     15004                 ARES               A CORUÑA               144   
4     15005              ARTEIXO               A CORUÑA               754   

                 
  Mujeres Total  
0     115   198  
1     889  1487  
2      21    55  
3     166   310  
4    1108  1862

In [384]:
df_paro_municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8134 entries, 0 to 8133
Data columns (total 6 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   (Municipio, Código)                  8134 non-null   int64 
 1   (Municipio, Nombre del municipio)    8134 non-null   object
 2   (Provincia, Nombre de la provincia)  8134 non-null   object
 3   (Número de parados, Hombres)         8134 non-null   int64 
 4   (Número de parados, Mujeres)         8134 non-null   int64 
 5   (Número de parados, Total)           8134 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 381.4+ KB


In [383]:
df_paro_municipios.to_csv("datos_limpiados/municipios/paro_municipios_limpio.csv", index=False)